# 1. V1 영상에 대해 preprocess + mtcnn + YOLO적용 후 영상 저장

In [ ]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from imageai.Detection import VideoObjectDetection
import os

cap = cv2.VideoCapture('videos/v1_raw.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 30

writer = cv2.VideoWriter('videos/v1.mp4', fourcc, fps, (width, height))
kernel = np.ones((2,2), np.uint8)

while True:
    ret,img = cap.read()

    if ret == False:
        break
    img = cv2.bilateralFilter(img, 9, 30, 30)
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    writer.write(img)
            
cap.release()
writer.release()
cv2.destroyAllWindows()


#######mtcnn적용#######

detector = MTCNN()
cap = cv2.VideoCapture('videos/v1.mp4')

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 30

writer = cv2.VideoWriter('videos/v1_mtcnn.mp4', fourcc, fps, (width, height))

while True: 
        
    ret, frame = cap.read()
    
    if ret == False: 
        break
        
    result = detector.detect_faces(frame)
    
    if result != []:
        for person in result:
            bounding_box = person['box']
            keypoints = person['keypoints']
    
            cv2.rectangle(frame,
                          (bounding_box[0], bounding_box[1]),
                          (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                          (0,155,255),
                          2)
    
            cv2.circle(frame,(keypoints['left_eye']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['right_eye']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['nose']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['mouth_left']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['mouth_right']), 2, (0,155,255), 2)
      
    writer.write(frame)
        
   
cap.release()
writer.release()
cv2.destroyAllWindows()

################yolo적용############

execution_path = os.getcwd()

detector = VideoObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath( os.path.join(execution_path , "models/yolo.h5"))
detector.loadModel()

video_path = detector.detectObjectsFromVideo(
    input_file_path=os.path.join(execution_path, "videos/v1_mtcnn.mp4"),
    output_file_path=os.path.join(execution_path, "videos/v1_mtcnn_YOLO"),
    frames_per_second=30, log_progress=True)
print(video_path)

# 2. mtcnn에서 얼굴 수 count후 파일 저장

In [ ]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from imageai.Detection import VideoObjectDetection
import os

detector = MTCNN()
cap = cv2.VideoCapture('videos/v1.mp4')

while True: 
    ret, frame = cap.read()
    
    if ret == False: 
        break

    file = open('videos/v1/face_count.txt', 'a')
    result = detector.detect_faces(frame)
    if result != []:
        for person in result:
            bounding_box = person['box']
            keypoints = person['keypoints']
    
            cv2.rectangle(frame,
                          (bounding_box[0], bounding_box[1]),
                          (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                          (0,155,255),
                          2)
    
            cv2.circle(frame,(keypoints['left_eye']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['right_eye']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['nose']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['mouth_left']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['mouth_right']), 2, (0,155,255), 2)
        file.write(str(len(result)) + '\n')       #result의 element수가 detect하는 수가 된다. 즉, 얼굴의 개수
    file.close()
   
cap.release()
writer.release()
cv2.destroyAllWindows()

# 3.YOLO에서 사람의 수 COUNT후 파일 저장

In [ ]:
from imageai.Detection import VideoObjectDetection
import os
from matplotlib import pyplot as plt

execution_path = os.getcwd()
resized = False

def forFrame(frame_number, output_array, output_count, returned_frame):
    counter = 0
    file = open('v1_person_count.txt', 'a')
    for eachItem in output_count:                            
        if eachItem == 'person':                           #eachItem이 사람이면 
            file.write(str(output_count[eachItem]) + '\n') #사람의 수를 file로 저장
    file.close()
    global resized

video_detector = VideoObjectDetection()
video_detector.setModelTypeAsYOLOv3()
video_detector.setModelPath(os.path.join(execution_path, "models/yolo.h5"))
video_detector.loadModel()

video_detector.detectObjectsFromVideo(input_file_path=os.path.join(execution_path, "videos/v1_mtcnn.mp4"), 
                                      output_file_path=os.path.join(execution_path, "videos/v1_yolo") ,  
                                      frames_per_second=30, per_frame_function=forFrame,  
                                      minimum_percentage_probability=50, 
                                      return_detected_frame=True)